In [1]:
# 웹드라이버 설정
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_autoinstaller
import webbrowser

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

#  그 외
import time
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

import re
import os

### 매일경제 - 경제, 국제

In [2]:
mks=['https://www.mk.co.kr/news/economy/', 'https://www.mk.co.kr/news/world/']

In [3]:
chromedriver_autoinstaller.install()
options = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=options)

# 저장할 딕셔너리
data = {'title':[], 'img':[], 'link':[]}

for mk in mks:
    driver.get(mk)
    time.sleep(1)

    # 스크롤 내리기
    last_height = driver.execute_script("return document.body.scrollHeight")
    SCROLL_PAUSE_TIME = 2
    while True:
        # 페이지 아래로 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 로딩 대기
        time.sleep(SCROLL_PAUSE_TIME)

        # 스크롤 후 높이 측정
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break  # 더 이상 스크롤 안 됨
        last_height = new_height

    # 뉴스 리스트 추출
    box_lists = driver.find_elements(By.CLASS_NAME,  'news_node')
    for list in box_lists:
        try: 
            # 1. 링크 추출
            link= list.find_element(By.TAG_NAME, 'a').get_attribute('href')
            print(link)

            # 2. 이미지 추출
            img_tag = list.find_element(By.TAG_NAME, 'img')
            img = img_tag.get_attribute('src') or img_tag.get_attribute('data-src') if img_tag else None
            print(img)

            # 3. 기사 제목 추출 
            title_el = list.find_element(By.CLASS_NAME, 'news_ttl')
            title = title_el.text if title_el else None
            print(title)

            if link and img and title:
                data['link'].append(link)
                data['title'].append(title)
                data['img'].append(img)

        except Exception as e:
            pass

driver.quit()

https://www.mk.co.kr/news/economy/11347228
https://wimg.mk.co.kr/news/cms/202506/19/news-p.v1.20250619.2a2047d870c54fcabd7ca94a15b9b0e5_P1.png
뉴스 쉽게보기
“자영업자 빚 탕감해 드려요”...새 정부가 추진하는 ‘배드 뱅크’
https://www.mk.co.kr/news/economy/11347728
https://wimg.mk.co.kr/news/cms/202506/20/news-p.v1.20250619.1610599f66194f2da32054c719ee23a8_P1.jpg
“부실채권 그렇게 팔았는데 왜”...카드사 연체율, 위험 수위 훌쩍 넘었다
https://www.mk.co.kr/news/stock/11347727
https://wimg.mk.co.kr/news/cms/202506/20/news-p.v1.20250619.4bd064659aa64f23a92d458617c5dd75_R.jpg
불공정거래 뿌리 뽑으려면…“미국처럼 금감원 파워 키워야”
https://www.mk.co.kr/news/economy/11347726
https://wimg.mk.co.kr/svc/desking/1000/index/202506/A11347726_29_20250620072802.png
재생에너지 키우려는 이재명 정부…원전 관련 예산은 편성 안해
https://www.mk.co.kr/news/economy/11347728
https://wimg.mk.co.kr/news/cms/202506/20/news-p.v1.20250619.d0aac409c58f43d9873091460b85756d_T1.png
“부실채권 그렇게 팔았는데 왜”...카드사 연체율, 위험 수위 훌쩍 넘었다
https://www.mk.co.kr/news/economy/11347228
https://wimg.mk.co.kr/news/cms/202506/19/news-p.v1.20250619.2d526

In [4]:
data

{'title': ['뉴스 쉽게보기\n“자영업자 빚 탕감해 드려요”...새 정부가 추진하는 ‘배드 뱅크’',
  '“부실채권 그렇게 팔았는데 왜”...카드사 연체율, 위험 수위 훌쩍 넘었다',
  '불공정거래 뿌리 뽑으려면…“미국처럼 금감원 파워 키워야”',
  '재생에너지 키우려는 이재명 정부…원전 관련 예산은 편성 안해',
  '“부실채권 그렇게 팔았는데 왜”...카드사 연체율, 위험 수위 훌쩍 넘었다',
  '“자영업자 빚 탕감해 드려요”...새 정부가 추진하는 ‘배드 뱅크’ 설립 [뉴스 쉽게보기]',
  '불공정거래 뿌리 뽑으려면…“미국처럼 금감원 파워 키워야”',
  '재생에너지 키우려는 이재명 정부...원전 관련 예산은 편성 안해',
  '“벌이는 줄었는데 씀씀이는 줄일 수가”...정부 총지출 첫 700조 돌파',
  '習·푸틴 통화서 중동 문제 논의 "이스라엘의 공격, 국제법 위반"',
  'K뷰티 ODM 산업…M&A 태풍의 눈',
  '시끌시끌 ‘초저가 한우’…초저가 언제까지?',
  '아기상어 그 회사…라이선스 상품 8천개 [IPO 기업 대해부]',
  '美 증시 불안에도 고공행진 미국판 ‘올영’ [미장 보석주]',
  '‘이란 핵포기’ 최종 시한 내건 트럼프 “2주내 군사 공격 여부 결정”',
  '“같이 공부하자”…여성들 자택으로 부르더니 中대학원생이 한 짓 ‘충격’',
  '美 “한국 등 아시아 동맹국, 국방비로 GDP 5% 수준 지출해야”',
  '‘이란 핵포기’ 최종 시한 내건 트럼프 “2주내 군사 공격 여부 결정”',
  '“우리 떨고 있는거니”...파열음 커지는 미국경제, 스태그플레이션 경고음',
  '베트남서 ‘전자담배’ 피우면 ‘25만원’ 벌금 폭탄',
  '이란 원전 폭발 땐 체르노빌급 재앙…오만·바레인서 피폭 대처법 확산',
  '“핸드폰 좀 열어봐”...유학비자 발급 절차 개시한 미국, SNS 검열 실시',
  '환경보호도 좋지만…‘이것’ 잘못쓰면 사망할 수도',
  '자국민 반대에도 “공격 준비 완료”...최종명령만 남은

In [5]:
df = pd.DataFrame(data)
df

,title,img,link
0,뉴스 쉽게보기\n“자영업자 빚 탕감해 드려요”...새 정부가 추진하는 ‘배드 뱅크’,https://wimg.mk.co.kr/news/cms/202506/19/news-...,https://www.mk.co.kr/news/economy/11347228
1,"“부실채권 그렇게 팔았는데 왜”...카드사 연체율, 위험 수위 훌쩍 넘었다",https://wimg.mk.co.kr/news/cms/202506/20/news-...,https://www.mk.co.kr/news/economy/11347728
2,불공정거래 뿌리 뽑으려면…“미국처럼 금감원 파워 키워야”,https://wimg.mk.co.kr/news/cms/202506/20/news-...,https://www.mk.co.kr/news/stock/11347727
3,재생에너지 키우려는 이재명 정부…원전 관련 예산은 편성 안해,https://wimg.mk.co.kr/svc/desking/1000/index/2...,https://www.mk.co.kr/news/economy/11347726
4,"“부실채권 그렇게 팔았는데 왜”...카드사 연체율, 위험 수위 훌쩍 넘었다",https://wimg.mk.co.kr/news/cms/202506/20/news-...,https://www.mk.co.kr/news/economy/11347728
5,“자영업자 빚 탕감해 드려요”...새 정부가 추진하는 ‘배드 뱅크’ 설립 [뉴스 쉽...,https://wimg.mk.co.kr/news/cms/202506/19/news-...,https://www.mk.co.kr/news/economy/11347228
6,불공정거래 뿌리 뽑으려면…“미국처럼 금감원 파워 키워야”,https://wimg.mk.co.kr/news/cms/202506/20/news-...,https://www.mk.co.kr/news/stock/11347727
7,재생에너지 키우려는 이재명 정부...원전 관련 예산은 편성 안해,https://wimg.mk.co.kr/news/cms/202506/20/news-...,https://www.mk.co.kr/news/economy/11347726
8,“벌이는 줄었는데 씀씀이는 줄일 수가”...정부 총지출 첫 700조 돌파,https://wimg.mk.co.kr/news/cms/202506/19/news-...,https://www.mk.co.kr/news/economy/11347708
9,"習·푸틴 통화서 중동 문제 논의 ""이스라엘의 공격, 국제법 위반""",https://wimg.mk.co.kr/news/cms/202506/20/20250...,https://www.mk.co.kr/news/economy/11347701


In [25]:
df.to_csv('./data/mk.csv', index=True)

### 한국경제 - 환율, 경제지표, 경제, 국제

In [6]:
한국경제 = 'https://www.hankyung.com/economy'

In [7]:
chromedriver_autoinstaller.install()
options = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=options)

driver.get(한국경제)
time.sleep(1)

# 환율
currencies = {'name':[], 'date':[], 'en': [], 'price':[], 'percent':[]}

# 경제지표
indicators = {'name':[], 'date':[], 'price':[]}


boxes = driver.find_elements(By.CLASS_NAME, 'data-list')
for box in boxes:
    # print(box.get_attribute('outerHTML'))
    a_tag = box.find_element(By.TAG_NAME, 'a')
    href = a_tag.get_attribute('href')
    if 'currencies' in href:
        name_cl = box.find_elements(By.CLASS_NAME, 'name')
        for nc in name_cl:
            name = nc.text.strip()
            currencies['name'].append(name)

        date_cl = box.find_elements(By.CLASS_NAME, 'txt-date')
        for dc in date_cl:
            date = dc.text.strip()
            currencies['date'].append(date)

        en_cl = box.find_elements(By.CLASS_NAME, 'txt-en')
        for ec in en_cl:
            en = ec.text.strip()
            currencies['en'].append(en)

        price_cl = box.find_elements(By.CLASS_NAME, 'price')
        for pc in price_cl:
            price = pc.text.strip()
            currencies['price'].append(price)
        
        percent_cl = box.find_elements(By.CLASS_NAME, 'percent')
        for pc in percent_cl:
            percent = pc.text.strip()
            currencies['percent'].append(percent)

        
    elif 'indicators' in href:
        name_cl = box.find_elements(By.CLASS_NAME, 'name')
        for nc in name_cl:
            name = nc.text.strip()
            indicators['name'].append(name)

        date_cl = box.find_elements(By.CLASS_NAME, 'txt-date')
        for dc in date_cl:
            date = dc.text.strip()
            indicators['date'].append(date)

        price_cl = box.find_elements(By.CLASS_NAME, 'price')
        for pc in price_cl:
            price = pc.text.strip()
            indicators['price'].append(price)        

In [8]:
currencies

{'name': ['미국', '일본', '중국', '유로', '홍콩'],
 'date': ['2025.06.20',
  '2025.06.20',
  '2025.06.20',
  '2025.06.20',
  '2025.06.20'],
 'en': ['USDKRW', 'JPYKRW', 'CNYKRW', 'EURKRW', 'HKDKRW'],
 'price': ['1,377.64', '948.37', '191.64', '1,584.48', '175.49'],
 'percent': ['-0.32%', '-0.02%', '-0.31%', '+0.02%', '-0.32%']}

In [9]:
df2 = pd.DataFrame(currencies)
df2

,name,date,en,price,percent
0,미국,2025.06.20,USDKRW,"1,377.64",-0.32%
1,일본,2025.06.20,JPYKRW,948.37,-0.02%
2,중국,2025.06.20,CNYKRW,191.64,-0.31%
3,유로,2025.06.20,EURKRW,"1,584.48",+0.02%
4,홍콩,2025.06.20,HKDKRW,175.49,-0.32%


In [26]:
df2.to_csv('./data/currencies.csv', index=True)

In [10]:
indicators

{'name': ['소비자물가지수', '경제성장률(GDP, 실질)', '실업률', '경상수지', '투자자 예탁금(백만원)'],
 'date': ['2025.05', '전년 대비', '2025.05', '2025.04', '2025.05'],
 'price': ['116.27', '1.40', '2.80', '5,701.70', '57,297,162.00']}

In [11]:
df3 = pd.DataFrame(indicators)
df3

,name,date,price
0,소비자물가지수,2025.05,116.27
1,"경제성장률(GDP, 실질)",전년 대비,1.40
2,실업률,2025.05,2.80
3,경상수지,2025.04,"5,701.70"
4,투자자 예탁금(백만원),2025.05,"57,297,162.00"


In [27]:
df3.to_csv('./data/indicators.csv', index=True)

In [12]:
한국경제_경제='https://www.hankyung.com/economy'

chromedriver_autoinstaller.install()
options = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=options)


driver.get(한국경제_경제)
time.sleep(1)

data = {'title':[], 'img':[], 'link':[]}

# 스크롤 내리기
last_height = driver.execute_script("return document.body.scrollHeight")
SCROLL_PAUSE_TIME = 2
while True:
    # 페이지 아래로 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 로딩 대기
    time.sleep(SCROLL_PAUSE_TIME)

    # 스크롤 후 높이 측정
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break  # 더 이상 스크롤 안 됨
    last_height = new_height


boxes = driver.find_elements(By.CSS_SELECTOR, '.module-row.col-line')
for box in boxes:
    name_cl = box.find_elements(By.CLASS_NAME, 'thumb')
    for nc in name_cl:
        img_tag = nc.find_element(By.TAG_NAME, 'img')
        img = img_tag.get_attribute('src') or img_tag.get_attribute('data-src') if img_tag else None
        name = img_tag.get_attribute('alt')
        link = nc.find_element(By.TAG_NAME, 'a').get_attribute('href')
        data['title'].append(name)
        data['img'].append(img)
        data['link'].append(link)

driver.quit()

In [13]:
df4 = pd.DataFrame(data)
df4

,title,img,link
0,4인 가족 '100만원 소비쿠폰'<br>고효율 가전 사면 10% 환급,https://img.hankyung.com/photo/202506/ZN.40764...,https://www.hankyung.com/article/2025061969271
1,'7년 넘은 5000만원 이하 빚' 탕감…113만명 수혜,https://img.hankyung.com/photo/202506/AA.40871...,https://www.hankyung.com/article/2025061968651
2,실업급여 예산 1.3兆 더 푼다…18만명 추가 혜택,https://img.hankyung.com/data/service/edit_img...,https://www.hankyung.com/article/202506195839i
3,올해 44兆 쏟아부어도 '0%대 성장' 우려,https://img.hankyung.com/photo/202506/AA.40871...,https://www.hankyung.com/article/2025061968661
4,"강민수 국세청장, 유럽 IOTA 총회 참석",https://img.hankyung.com/photo/202506/AA.40874...,https://www.hankyung.com/article/2025061968451
5,"이재명 대통령 ""이제 국가재정 쓸 때""…30.5조 추경 속도전",https://img.hankyung.com/photo/202506/AA.40874...,https://www.hankyung.com/article/2025061969471
6,'한우가 와규보다 식감 좋고 맛있어요'…중동서 '인기 폭발',https://img.hankyung.com/photo/202506/01.40869...,https://www.hankyung.com/article/202506196044i
7,"이창용 ""추경, 물가에 주는 영향 크지 않을 것""",https://img.hankyung.com/photo/202506/AA.40862...,https://www.hankyung.com/article/2025061842751
8,대학생 '천원 아침밥' 부러웠는데…직장인들도 '희소식',https://img.hankyung.com/photo/202506/AA.40862...,https://www.hankyung.com/article/2025061842711
9,소비자웰빙환경만족지수 1위 인증 수여식,https://img.hankyung.com/photo/202506/AA.40861...,https://www.hankyung.com/article/2025061841441


In [14]:
한국경제_국제='https://www.hankyung.com/international'

chromedriver_autoinstaller.install()
options = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=options)


driver.get(한국경제_국제)
time.sleep(1)

data = {'title':[], 'img':[], 'link':[]}

# 스크롤 내리기
last_height = driver.execute_script("return document.body.scrollHeight")
SCROLL_PAUSE_TIME = 2
while True:
    # 페이지 아래로 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 로딩 대기
    time.sleep(SCROLL_PAUSE_TIME)

    # 스크롤 후 높이 측정
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break  # 더 이상 스크롤 안 됨
    last_height = new_height

container = driver.find_element(By.ID, 'container')
top_wrap = container.find_elements(By.CLASS_NAME, 'module-row')
for tw in top_wrap:
    thumb_cl = tw.find_elements(By.CLASS_NAME, 'thumb')
    for tc in thumb_cl:
        link= tc.find_element(By.TAG_NAME, 'a').get_attribute('href')
        img_tag = tc.find_element(By.TAG_NAME, 'img')
        img = img_tag.get_attribute('src') or img_tag.get_attribute('data-src') if img_tag else None
        title = img_tag.get_attribute('alt')
        data['title'].append(title)
        data['img'].append(img)
        data['link'].append(link)
    
contents = container.find_elements(By.CSS_SELECTOR, 'cont-wrap>contents>module-row')
for ct in contents:
    informations = ct.find_elements(By.CLASS_NAME, 'thumb')
    for info in informations:
        link = info.find_element(By.TAG_NAME, 'a').get_attribute('href')
        img_tag = info.find_element(By.TAG_NAME, 'img')
        img = img_tag.get_attribute('src') or img_tag.get_attribute('data-src') if img_tag else None
        title = img_tag.get_attribute('alt')
        data['title'].append(title)
        data['img'].append(img)
        data['link'].append(link)

In [15]:
data

{'title': ['이란 공격 결정 미루는 트럼프<br>"장고 뒤엔 이라크戰 악몽"',
  "이스라엘, '핵무기 제조 원천' 이란 원자로 때렸다",
  '텍사스인스트루먼트, 美 반도체 공장에 600억弗 투자',
  '"치킨값 또 오르겠네" 우려 쏟아졌는데…드디어 끝났다',
  '똑같이 생긴 절친 여고생…알고 보니 헤어진 쌍둥이였다',
  '이스라엘 "이란, 집속탄 장착 탄도미사일로 공격"',
  '"비자 절차 재개" 美 유학생 부모 한숨 돌렸는데…\'논란 폭발\' 왜?',
  "'까딱하면 트럼프 때문에 美 경제 박살'…심상치 않은 경고",
  '"쾅!" 폭발한 스페이스X 로켓…순식간에 불기둥 치솟았다',
  "'트럼프는 독재자'…워싱턴 한복판에 등장한 조각상 정체",
  '\'중증 치매\' 브루스 윌리스 향한 딸의 고백…"모든 순간 감사"',
  '"비자 절차 재개" 美 유학생 부모 한숨 돌렸는데…\'논란 폭발\' 왜?',
  "'트럼프는 독재자'…워싱턴 한복판에 등장한 조각상 정체",
  '\'중증 치매\' 브루스 윌리스 향한 딸의 고백…"모든 순간 감사"',
  '"치킨값 또 오르겠네" 우려 쏟아졌는데…드디어 끝났다',
  '브라질 닭고기 이르면 이달 말 수입 재개 예정',
  "'암 극복' 은퇴 男, 9개월 동안 복권 3번 당첨…34억 '잭팟'",
  "캐나다 국방지출 GDP 2%로 늘린다…'K-방산' 또 '대박'?",
  "콜롬비아 대선주자, 유세 중 총기 피격…'15세 용의자' 체포",
  '브라질 닭고기 이르면 이달 말 수입 재개 예정',
  "'암 극복' 은퇴 男, 9개월 동안 복권 3번 당첨…34억 '잭팟'",
  '이시바 "한일, 협력할 분야 많아…젊은 세대 교류, 밝은 미래 느껴"',
  '日이시바 "젊은 세대 교류에서 밝은 미래 느껴"',
  '이시바 "李대통령과 첫 만남서 생각 맞추며 좋은 논의"',
  '이시바 "한일, 공통과제 직면…협력할 분야 많아"',
  "이시바, 韓주최 도쿄 '한일수교 60주년 행사' 참석",
  "22년간 화

In [16]:
df5 = pd.DataFrame(data)
df5

,title,img,link
0,"이란 공격 결정 미루는 트럼프<br>""장고 뒤엔 이라크戰 악몽""",https://img.hankyung.com/photo/202506/AA.40871...,https://www.hankyung.com/article/2025061968701
1,"이스라엘, '핵무기 제조 원천' 이란 원자로 때렸다",https://img.hankyung.com/photo/202506/AA.40871...,https://www.hankyung.com/article/2025061968711
2,"텍사스인스트루먼트, 美 반도체 공장에 600억弗 투자",https://img.hankyung.com/data/service/edit_img...,https://www.hankyung.com/article/2025061968721
3,"""치킨값 또 오르겠네"" 우려 쏟아졌는데…드디어 끝났다",https://img.hankyung.com/photo/202506/99.40542...,https://www.hankyung.com/article/2025061947957
4,똑같이 생긴 절친 여고생…알고 보니 헤어진 쌍둥이였다,https://img.hankyung.com/photo/202506/01.40866...,https://www.hankyung.com/article/2025061953457
5,"이스라엘 ""이란, 집속탄 장착 탄도미사일로 공격""",https://img.hankyung.com/photo/202506/ZA.40874...,https://www.hankyung.com/article/2025061972817
6,"""비자 절차 재개"" 美 유학생 부모 한숨 돌렸는데…'논란 폭발' 왜?",https://img.hankyung.com/photo/202506/ZA.40839...,https://www.hankyung.com/article/2025061947217
7,'까딱하면 트럼프 때문에 美 경제 박살'…심상치 않은 경고,https://img.hankyung.com/photo/202506/99.18939...,https://www.hankyung.com/article/2025061821727
8,"""쾅!"" 폭발한 스페이스X 로켓…순식간에 불기둥 치솟았다",https://img.hankyung.com/photo/202506/ZA.40872...,https://www.hankyung.com/article/2025062074347
9,'트럼프는 독재자'…워싱턴 한복판에 등장한 조각상 정체,https://img.hankyung.com/photo/202506/ZA.40862...,https://www.hankyung.com/article/2025061972197


In [17]:
df6 = pd.concat([df4, df5],axis=0, ignore_index=True)
df6

,title,img,link
0,4인 가족 '100만원 소비쿠폰'<br>고효율 가전 사면 10% 환급,https://img.hankyung.com/photo/202506/ZN.40764...,https://www.hankyung.com/article/2025061969271
1,'7년 넘은 5000만원 이하 빚' 탕감…113만명 수혜,https://img.hankyung.com/photo/202506/AA.40871...,https://www.hankyung.com/article/2025061968651
2,실업급여 예산 1.3兆 더 푼다…18만명 추가 혜택,https://img.hankyung.com/data/service/edit_img...,https://www.hankyung.com/article/202506195839i
3,올해 44兆 쏟아부어도 '0%대 성장' 우려,https://img.hankyung.com/photo/202506/AA.40871...,https://www.hankyung.com/article/2025061968661
4,"강민수 국세청장, 유럽 IOTA 총회 참석",https://img.hankyung.com/photo/202506/AA.40874...,https://www.hankyung.com/article/2025061968451
5,"이재명 대통령 ""이제 국가재정 쓸 때""…30.5조 추경 속도전",https://img.hankyung.com/photo/202506/AA.40874...,https://www.hankyung.com/article/2025061969471
6,'한우가 와규보다 식감 좋고 맛있어요'…중동서 '인기 폭발',https://img.hankyung.com/photo/202506/01.40869...,https://www.hankyung.com/article/202506196044i
7,"이창용 ""추경, 물가에 주는 영향 크지 않을 것""",https://img.hankyung.com/photo/202506/AA.40862...,https://www.hankyung.com/article/2025061842751
8,대학생 '천원 아침밥' 부러웠는데…직장인들도 '희소식',https://img.hankyung.com/photo/202506/AA.40862...,https://www.hankyung.com/article/2025061842711
9,소비자웰빙환경만족지수 1위 인증 수여식,https://img.hankyung.com/photo/202506/AA.40861...,https://www.hankyung.com/article/2025061841441


In [29]:
df6.to_csv('./data/hk.csv', index=True)

### 연합뉴스TV - 경제, 세계

In [18]:
yonhapnews=['https://www.yonhapnewstv.co.kr/news?ct=3', 'https://www.yonhapnewstv.co.kr/news?ct=6']

In [19]:
chromedriver_autoinstaller.install()
options = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=options)

# 저장할 딕셔너리
data = {'title':[], 'img':[], 'link':[]}

for yh in yonhapnews:
    driver.get(yh)
    time.sleep(1)

    # 스크롤 내리기
    last_height = driver.execute_script("return document.body.scrollHeight")
    SCROLL_PAUSE_TIME = 2
    while True:
        # 페이지 아래로 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 로딩 대기
        time.sleep(SCROLL_PAUSE_TIME)

        # 스크롤 후 높이 측정
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break  # 더 이상 스크롤 안 됨
        last_height = new_height



    content = driver.find_element(By.ID, 'content')
    conts = content.find_elements(By.CLASS_NAME, 'grid-col')
    for cont in conts:
        cont_a = cont.find_element(By.TAG_NAME, 'a')
        link = cont_a.get_attribute('href')
        img_cl = cont.find_element(By.CLASS_NAME, 'item-img')
        img_tag = img_cl.find_element(By.TAG_NAME, 'img')
        img = img_tag.get_attribute('src') or img_tag.get_attribute('data-src') if img_tag else None
        title = img_tag.get_attribute('alt')
        
        data['title'].append(title)
        data['img'].append(img)
        data['link'].append(link)


    cont_item = content.find_element(By.CLASS_NAME, 'article-webzine')
    items = cont_item.find_elements(By.CLASS_NAME, 'item')
    for item in items:
        try:
            img_tag = item.find_element(By.TAG_NAME, 'img')
            img = img_tag.get_attribute('src') or img_tag.get_attribute('data-src') if img_tag else None
            title_tag = item.find_element(By.CLASS_NAME, 'title')
            link = title_tag.get_attribute('href')
            title = title_tag.text.strip()
            if title is False:
                title =  img_tag.get_attribute('alt')
            
            data['title'].append(title)
            data['img'].append(img)
            data['link'].append(link)

        except Exception as e:
            print('에러:', e)

driver.quit()

In [20]:
data

{'title': ["2차 추경 '20조원' 푼다…",
  "생산자물가 두 달 연속 하락…농산물 '뚝'",
  '김병주 MBK 회장, 국회 찾아 사재 출연 거부 표명',
  "코스피, 나흘째 상승…추경에 '3천피' 가나",
  "소상공인 113만명 '빚 탕감'…'배드뱅크' 설립",
  '기초연금 부부감액 20%…"대체로 적정하지만 취약계층은 불리"',
  '"모델하우스와 너무 달라"…아파트 소비자 상담 급증',
  "클릭도 안했는데 또 쿠팡으로?…방통위, '납치광고' 조사 착수",
  'MSCI, 한국 증시 공매도 개선…"외국인 투자 여전히 제약"',
  '중소SO-CJ ENM, 상생발전 위한 업무 협약',
  "AI수석 하정우, '천만배우' 하정우 인지도 제쳤다",
  '"상반기 할인 막차타세요"…신세계·롯데 쇼핑 행사 가득',
  "생산자물가 두 달 연속 하락…농산물 '뚝'",
  '5월 생산자물가 두 달 연속 하락…농산물 10.1% 내려',
  '"장마철 특수 잡아라"…유통가, 여름 마케팅 돌입',
  '김병주 MBK 회장, 국회 찾아 사재 출연 거부 표명',
  '여름철 인기 \'간편식 냉면\'…"나트륨 과다 섭취 주의해야"',
  "코스피, 나흘째 상승…추경에 '3천피' 가나",
  "소상공인 113만명 '빚 탕감'…'배드뱅크' 설립",
  '2차 추경 \'20조원\' 푼다…"4인가족 최대 200만원"',
  '여름 장마 본격화…운전자 안전 대처 요령은?',
  '토스뱅크서 20억원대 횡령 사고…"고객 피해 없어"',
  '전공의 200여명 "복귀 희망"…이번엔 진짜 돌아오나',
  '"혼자 살아요" 800만…맞벌이 가구는 4년만에 감소',
  '금융위, DSR 규제 확대 고민…가상자산ETF 도입 마련',
  "서울 아파트값 '불장' 재현…6년 9개월만 최대폭 상승",
  '타국 실권자에 자국 사령관 험담...태국 총리 통화 유출 파문',
  '',
  "인도 '여객기 추락' 유일 생존자 ",
  '모사드, 대이란 심리전 강화…이란은 첩자 색출 총력전

In [21]:
df7 = pd.DataFrame(data)
df7

,title,img,link
0,2차 추경 '20조원' 푼다…,https://yonhapnewstv-prod.s3.ap-northeast-2.am...,https://www.yonhapnewstv.co.kr/news/MYH2025061...
1,생산자물가 두 달 연속 하락…농산물 '뚝',https://yonhapnewstv-prod.s3.ap-northeast-2.am...,https://www.yonhapnewstv.co.kr/news/MYH2025062...
2,"김병주 MBK 회장, 국회 찾아 사재 출연 거부 표명",https://yonhapnewstv-prod.s3.ap-northeast-2.am...,https://www.yonhapnewstv.co.kr/news/MYH2025062...
3,"코스피, 나흘째 상승…추경에 '3천피' 가나",https://yonhapnewstv-prod.s3.ap-northeast-2.am...,https://www.yonhapnewstv.co.kr/news/MYH2025061...
4,소상공인 113만명 '빚 탕감'…'배드뱅크' 설립,https://yonhapnewstv-prod.s3.ap-northeast-2.am...,https://www.yonhapnewstv.co.kr/news/MYH2025061...
5,"기초연금 부부감액 20%…""대체로 적정하지만 취약계층은 불리""",https://yonhapnewstv-prod.s3.ap-northeast-2.am...,https://www.yonhapnewstv.co.kr/news/AKR2025062...
6,"""모델하우스와 너무 달라""…아파트 소비자 상담 급증",https://yonhapnewstv-prod.s3.ap-northeast-2.am...,https://www.yonhapnewstv.co.kr/news/AKR2025062...
7,"클릭도 안했는데 또 쿠팡으로?…방통위, '납치광고' 조사 착수",https://yonhapnewstv-prod.s3.ap-northeast-2.am...,https://www.yonhapnewstv.co.kr/news/AKR2025062...
8,"MSCI, 한국 증시 공매도 개선…""외국인 투자 여전히 제약""",https://yonhapnewstv-prod.s3.ap-northeast-2.am...,https://www.yonhapnewstv.co.kr/news/AKR2025062...
9,"중소SO-CJ ENM, 상생발전 위한 업무 협약",https://yonhapnewstv-prod.s3.ap-northeast-2.am...,https://www.yonhapnewstv.co.kr/news/AKR2025062...


In [30]:
df7.to_csv('./data/yh.csv', index=True)